# Web Scraping TAZ.de (Artikel & Kommentare) mit `requests` & `BeautifulSoup`

Dieses Notebook demonstriert, wie man Artikeldetails und Kommentare von der Nachrichtenseite TAZ.de extrahieren kann, **ohne** auf Browser-Automatisierungstools wie Selenium angewiesen zu sein. Dies ist möglich, da TAZ.de viele seiner Inhalte, einschließlich der Kommentare, oft direkt im initialen HTML-Quelltext ausliefert (serverseitiges Rendering).

**Ziele:**
1.  Artikel-Metadaten (Titel, Kicker, Datum, etc.) extrahieren.
2.  Artikel-Volltext extrahieren.
3.  Kommentare, einschließlich verschachtelter Antworten, extrahieren und zählen.
4.  Die Daten strukturiert speichern.

**Verwendete Bibliotheken:**
*   `requests`: Zum Abrufen des HTML-Codes der Webseite.
*   `BeautifulSoup`: Zum Parsen des HTML-Codes und Extrahieren der Daten.
*   `json`: Zum Speichern der Ergebnisse.
*   `time`: Für Pausen (optional, aber empfohlen).

**Wichtiger Hinweis:** Das Funktionieren dieses Skripts hängt von der aktuellen HTML-Struktur von TAZ.de ab. Webseiten ändern sich häufig, daher müssen die verwendeten Selektoren (z.B. `class_=` Namen) möglicherweise in Zukunft angepasst werden.

## Teil 0: Setup

Installation (falls nötig) und Import der Bibliotheken.

In [ ]:
# Installieren der Bibliotheken (normalerweise in Colab vorinstalliert)
# !pip install requests beautifulsoup4 lxml

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import random # Für optional zufällige Pausen

## Teil 1: Funktion zur Extraktion von Kommentaren (inkl. Subkommentare)

Wir definieren zuerst eine Hilfsfunktion, die rekursiv die Anzahl der Antworten (Subkommentare) auf einen Kommentar zählt. Dies basiert auf der Struktur Ihres ursprünglichen Skripts.

In [ ]:
# Function to recursively count subcomments (based on the provided taz-scraper code)
def count_subcomments(element):
    """Zählt rekursiv die Anzahl der direkten und indirekten Subkommentare."""
    subcomments_count = 0
    if element:
        # Finde direkte Kind-Kommentare (li mit Klasse 'member')
        subcomments = element.find_all('li', class_='member', recursive=False)
        subcomments_count += len(subcomments)
        # Für jeden direkten Subkommentar...
        for subcomment in subcomments:
            # ...finde dessen Container für weitere Subkommentare...
            sub_subcomments_list = subcomment.find('ul', class_='thread')
            # ...und rufe die Funktion rekursiv auf.
            subcomments_count += count_subcomments(sub_subcomments_list)
    return subcomments_count

## Teil 2: Funktion zum Scrapen eines einzelnen TAZ-Artikels

Diese Funktion nimmt eine URL entgegen, ruft die Seite ab, parst das HTML und extrahiert die gewünschten Daten.

In [ ]:
def scrape_taz_article(url, headers):
    """
    Scrapt Details und Kommentare von einer gegebenen TAZ-Artikel-URL.

    Args:
        url (str): Die URL des TAZ-Artikels.
        headers (dict): HTTP-Header für die Anfrage (insb. User-Agent).

    Returns:
        dict: Ein Dictionary mit den gescrapten Daten oder None bei einem Fehler.
    """
    try:
        print(f"  Versuche URL abzurufen: {url}")
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status() # Fehler bei nicht-200 Status

        print(f"  Status Code: {response.status_code}")
        soup = BeautifulSoup(response.content, 'lxml') # 'lxml' ist meist schneller

        # --- Artikelinformationen extrahieren ---
        # Selektoren basieren auf dem bereitgestellten Skript - können sich ändern!
        kicker_tag = soup.find('span', class_='kicker')
        kicker = kicker_tag.get_text(strip=True) if kicker_tag else 'N/A'

        # Titel kann in mehreren Spans sein, nimm den letzten
        title = 'N/A'
        title_h1 = soup.find('h1', itemprop='headline')
        if title_h1:
           title_spans = title_h1.find_all('span')
           if title_spans:
               title = title_spans[-1].get_text(strip=True)


        summary_tag = soup.find('p', itemprop='description')
        summary = summary_tag.get_text(strip=True) if summary_tag else 'N/A'

        # Zeitstempel aus 'content'-Attribut extrahieren
        time_tag = soup.find('li', itemprop='datePublished')
        timestamp_iso = time_tag['content'] if time_tag and time_tag.has_attr('content') else 'N/A'


        # Artikeltext - alle <p> mit Klasse 'article' zusammenfügen
        article_p_tags = soup.find_all('p', class_='article')
        article_content = ' '.join([p.get_text(strip=True) for p in article_p_tags])
        if not article_content: # Fallback falls Klasse nicht (mehr) existiert
            # Versuche einen allgemeineren Container zu finden (muss ggf. angepasst werden)
            article_body = soup.find('div', class_='sectbody')
            if article_body:
                 article_p_tags = article_body.find_all('p')
                 article_content = ' '.join([p.get_text(strip=True) for p in article_p_tags])
            else:
                 article_content = "Artikeltext nicht gefunden (Selektor prüfen!)"


        # --- Kommentare extrahieren ---
        comments_data = []
        # Finde alle Top-Level Kommentare (li mit Klasse 'member', aber nicht in einem 'ul.thread')
        top_level_comments = soup.select('ul.comments > li.member') # CSS-Selektor für Top-Level

        print(f"  Anzahl Top-Level Kommentare gefunden: {len(top_level_comments)}")

        for comment in top_level_comments:
            name = 'N/A'
            name_h4 = comment.find('h4')
            if name_h4:
                name = name_h4.get_text(strip=True)

            profile_link = 'N/A'
            author_link = comment.find('a', class_='author person')
            if author_link and author_link.has_attr('href'):
                profile_link = 'https://taz.de' + author_link['href']

            comment_time = 'N/A'
            time_tag_comment = comment.find('time')
            if time_tag_comment and time_tag_comment.has_attr('datetime'):
                comment_time = time_tag_comment['datetime']

            # Kommentartext - kann in mehreren <p> sein
            comment_text_parts = comment.find_all('p', class_=lambda x: x in x.split()) # Findet <p class="odd">, <p class="even"> etc.
            if not comment_text_parts: # Fallback, falls Klassenstruktur anders ist
                 comment_body_div = comment.find('div', class_='comment_text') # Beispiel für alternativen Selektor
                 if comment_body_div:
                     comment_text_parts = comment_body_div.find_all('p')

            comment_text = ' '.join(p.get_text(strip=True) for p in comment_text_parts)

            # Check for edits by 'taz' and remove the edit note from the comment text (optional)
            # edited_by_taz = any('taz' in p.get_text() for p in comment_text_parts)
            # if edited_by_taz:
            #     comment_text = comment_text.replace('taz: "Taz comment added in"', '').strip() # Beispiel-Text anpassen

            # Finde den Container für Subkommentare dieses Kommentars
            subcomments_list_ul = comment.find('ul', class_='thread')
            response_comments_count = count_subcomments(subcomments_list_ul)

            comments_data.append({
                'Name': name,
                'Profile link': profile_link,
                'Timestamp': comment_time,
                'Comment text': comment_text,
                'Response comments count': response_comments_count,
            })

        # --- Zusammenstellen der Artikeldaten ---
        article_data = {
            'URL': url,
            'Kicker': kicker,
            'Title': title,
            'Summary': summary,
            'Time': timestamp_iso,
            'Article Text': article_content,
            'Comments': comments_data,
            'TopLevelCommentCount': len(top_level_comments) # Anzahl direkter Kommentare
        }
        print(f"  Artikel erfolgreich verarbeitet.")
        return article_data

    except requests.exceptions.RequestException as e:
        print(f"  Fehler beim Abrufen der URL {url}: {e}")
        return None
    except Exception as e:
        print(f"  Unerwarteter Fehler beim Verarbeiten von {url}: {e}")
        return None

## Teil 3: Ausführung des Scrapings für Beispiel-URLs

Wir definieren eine Liste von Beispiel-TAZ-URLs und wenden unsere Scraping-Funktion darauf an.

In [ ]:
# Beispiel TAZ URLs (ersetzen oder erweitern Sie diese Liste)
# Suchen Sie sich einige aktuelle TAZ-Artikel mit Kommentaren heraus.
example_urls = [
    "https://taz.de/Gladbecker-Geiseldrama-vor-35-Jahren/!5949917/", # Beispiel älter
    "https://taz.de/Debatte-ueber-Direktkandidaturen/!5997264/",     # Beispiel neuer
    "https://taz.de/FDP-regiert-im-Bund-mit/!5997279/",              # Beispiel mit anderer Struktur?
    "https://taz.de/!5997684/"                                        # Beispiel ohne Kommentare?
    # Fügen Sie hier weitere TAZ URLs hinzu
]

# Wichtig: User-Agent setzen!
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

all_scraped_data = []
MAX_ARTICLES_DEMO = len(example_urls) # Scrapen aller Beispiel-URLs

print(f"Starte Scraping für {MAX_ARTICLES_DEMO} Beispiel-Artikel...")

for i, url in enumerate(example_urls):
    print(f"\nVerarbeite Artikel {i+1}/{MAX_ARTICLES_DEMO}...")
    article_data = scrape_taz_article(url, headers)

    if article_data:
        all_scraped_data.append(article_data)

    # Optionale Pause zwischen den Anfragen
    pause = random.uniform(1.5, 3.5)
    print(f"  Pause für {pause:.1f} Sekunden...")
    time.sleep(pause)

print(f"\nScraping abgeschlossen. {len(all_scraped_data)} Artikel erfolgreich verarbeitet.")

## Teil 4: Ergebnisse anzeigen und speichern

In [ ]:
# Zeige die Daten des ersten erfolgreich gescrapten Artikels (falls vorhanden)
if all_scraped_data:
    print("\n--- Beispiel Daten (erster Artikel) ---")
    # Pretty print using json.dumps for better readability
    print(json.dumps(all_scraped_data[0], indent=2, ensure_ascii=False))
else:
    print("\nKeine Daten zum Anzeigen vorhanden.")

In [ ]:
# Ergebnisse in einer JSON-Datei speichern
import datetime
output_filename_taz = f"taz_scraped_data_demo_{datetime.date.today()}.json"

try:
    with open(output_filename_taz, 'w', encoding='utf-8') as f:
        json.dump(all_scraped_data, f, ensure_ascii=False, indent=4)
    print(f"\nErgebnisse wurden erfolgreich in '{output_filename_taz}' gespeichert.")

    # Optional: Datei im Colab herunterladen
    # from google.colab import files
    # files.download(output_filename_taz)

except Exception as e:
    print(f"\nFehler beim Speichern der Datei: {e}")

## Fazit & Nächste Schritte

Dieses Notebook hat gezeigt, dass es für bestimmte Webseiten wie TAZ.de möglich ist, auch komplexe Daten wie Kommentare nur mit `requests` und `BeautifulSoup` zu extrahieren.

**Wichtige Punkte:**
*   **Abhängigkeit von der Seitenstruktur:** Die verwendeten Selektoren (`find()`, `find_all()`, `select()`) sind spezifisch für die TAZ-Struktur zum Zeitpunkt der Erstellung. Änderungen an der Webseite können das Skript unbrauchbar machen. Regelmäßige Überprüfung und Anpassung sind notwendig.
*   **Fehlerbehandlung:** Eine robuste Implementierung sollte mehr Fehlerbehandlung enthalten (z.B. wenn bestimmte Elemente nicht gefunden werden).
*   **Ethik & Recht:** Auch hier gelten die Regeln für verantwortungsvolles Scraping (siehe `session2.html`): Nutzungsbedingungen prüfen, Server nicht überlasten (Pausen!), Datenschutz beachten.

Sie können dieses Notebook als Grundlage verwenden und anpassen, um eine größere Anzahl von TAZ-Artikeln zu scrapen (z.B. indem Sie eine längere Liste von URLs bereitstellen).